In [16]:
import sqlite3
import mysql.connector

In [ ]:
import sqlite3

conn = sqlite3.connect('mysqlite.db')
c = conn.cursor()
#records or rows in a list
records = [(1, 'Glen', 8),
			(2, 'Elliot', 9),
			(3, 'Bob', 7)]

#insert multiple records in a single query
c.executemany('INSERT INTO students VALUES(?,?,?);',records);

print('We have inserted', c.rowcount, 'records to the table.')

#commit the changes to db			
conn.commit()
#close the connection
conn.close()


## remove not downloaded ads 

In [163]:
color = ['Red', 'Green', 'White', 'Black', 'Pink', 'Yellow']
with open('file1.txt','w+') as f:
	for i in color:
		f.write('%s,'%i)

# with open('file1.txt') as f:
# 	print(f.read())
###############
list1=[]
with open('file1.txt','r+') as f:
	lines = f.readlines()
list1 = lines[0].split(',')
list1.remove('')
list1

['Red', 'Green', 'White', 'Black', 'Pink', 'Yellow']

In [88]:
to_remove_list = []
mydb = mysql.connector.connect(
            host="database-1.cclpp8z8lgmz.us-east-1.rds.amazonaws.com",
            user="admin",
            password="AWS$2020",
            port = 3306, #for Mamp users
            database='facebook_ads'
            )
mycursor = mydb.cursor()
sqlite_select_query = f"""select ad_id ,videos,poster from ads;"""
mycursor.execute(sqlite_select_query)
records = mycursor.fetchall()
mycursor.close()
mydb.close()
for record in records:
    videos = record[1].split()
    posters = record[2].split()
    i = 0
    for video in videos:
        name = re.findall('\d+_\d+_\d+_n',video)[0] + '.mp4'
        data = cv2.VideoCapture(video)
        frames = data.get(cv2.CAP_PROP_FRAME_COUNT)
        fps = data.get(cv2.CAP_PROP_FPS)
        seconds = round(frames / fps)
        if seconds < 120:
            import os
            if not os.path.exists(f'./media/vids/{name}'):
                try:
                    urllib.request.urlretrieve(video, f'./media/vids/{name}')
                except:
                    to_remove_list.append(record[0])
                try:
                    poster = posters[i]
                    i += 1
                    poster_name = re.findall('\d+_\d+_\d+_n',poster)[0] + '.jpg'
                    img_data = requests.get(poster).content
                    with open(f'./media/pics/{poster_name}', 'wb') as handler:
                        handler.write(img_data)
                except:
                    print('poster broken for ad',record[0])  
print(len(to_remove_list))
with open('to_remove_list.txt','w+') as f:
    for i in to_remove_list:
        f.write('%s,'%i)

In [ ]:
with open('to_remove_list.txt','r+') as f:
	lines = f.readlines()
to_remove_list = lines[0].split(',')
to_remove_list.remove('')
mydb = mysql.connector.connect(
            host="database-1.cclpp8z8lgmz.us-east-1.rds.amazonaws.com",
            user="admin",
            password="AWS$2020",
            port = 3306, #for Mamp users
            database='facebook_ads'
            )
mycursor = mydb.cursor()
sqlite_select_query = f"""DELETE FROM ads WHERE (ad_id IN ({",".join(["'"+str(x)+"'" for x in to_remove_list])}));"""
mycursor.execute(sqlite_select_query)
# records = mycursor.fetchall()
mydb.commit()
mycursor.close()
mydb.close()

### migrate all the data into production 

In [ ]:
mydb = mysql.connector.connect(
            host="database-1.cclpp8z8lgmz.us-east-1.rds.amazonaws.com",
            user="admin",
            password="AWS$2020",
            port = 3306, #for Mamp users
            database='demo'
            )
mycursor = mydb.cursor()
sqlite_select_query = f"""select * from facebook_pages;"""
mycursor.execute(sqlite_select_query)
records1 = mycursor.fetchall()
mycursor.close()
mydb.close()

sqliteConnection = sqlite3.connect('FaceBoookADS.db')
cursor = sqliteConnection.cursor()
sqlite_select_query = f"""select * from facebook_pages;"""
cursor.execute(sqlite_select_query)
records2 = cursor.fetchall()
cursor.close()
sqliteConnection.close()

records = list(set(records1+records2))


In [ ]:
mydb = mysql.connector.connect(
            host="database-1.cclpp8z8lgmz.us-east-1.rds.amazonaws.com",
            user="admin",
            password="AWS$2020",
            port = 3306, #for Mamp users
            database='demo'
            )
mycursor = mydb.cursor()
sqlite_select_query = f"""select * from facebook_pages limit 1;"""
mycursor.execute(sqlite_select_query)
records1 = mycursor.fetchall()
mycursor.close()
mydb.close()

sqliteConnection = sqlite3.connect('FaceBoookADS.db')
cursor = sqliteConnection.cursor()
sqlite_select_query = f"""select * from facebook_pages limit 1;"""
cursor.execute(sqlite_select_query)
records2 = cursor.fetchall()
cursor.close()
sqliteConnection.close()

In [59]:
table_name = 'search_ter'
mydb = mysql.connector.connect(
            host="database-1.cclpp8z8lgmz.us-east-1.rds.amazonaws.com",
            user="admin",
            password="AWS$2020",
            port = 3306, #for Mamp users
            database='facebook_ads'
            )

sqlite_select_query = f'''SELECT `COLUMN_NAME` 
FROM `INFORMATION_SCHEMA`.`COLUMNS` 
WHERE `TABLE_SCHEMA`='facebook_ads' 
    AND `TABLE_NAME`='{table_name}';'''

mycursor = mydb.cursor()
mycursor.execute(sqlite_select_query)
order_cols = mycursor.fetchall()
order_cols = [item for sublist in order_cols for item in sublist]
order_cols.remove('state')
mycursor.close()
mydb.close()
print(order_cols)

ValueError: list.remove(x): x not in list

In [75]:
sql_txt = f"INSERT INTO {table_name}({','.join(order_cols)}) VALUES ({','.join(['%s']*5)})"
sql_txt

'INSERT INTO search_terms(search_term,search_type,static_ID,country,active,last_visited) VALUES (%s,%s,%s,%s,%s)'

In [73]:
f"""select {','.join(order_cols)} from {table_name};"""

'select search_term,search_type,static_ID,country,active,last_visited from search_terms;'

In [80]:
mydb = mysql.connector.connect(
            host="database-1.cclpp8z8lgmz.us-east-1.rds.amazonaws.com",
            user="admin",
            password="AWS$2020",
            port = 3306, #for Mamp users
            database='Demo'
            )
mycursor = mydb.cursor()
sqlite_select_query = f"""select * from {table_name} limit 5;"""
mycursor.execute(sqlite_select_query)
records = mycursor.fetchall()
mycursor.close()
mydb.close()

In [78]:
import sqlite3
import mysql.connector
#get the right order
table_name = 'ads'
mydb = mysql.connector.connect(
            host="database-1.cclpp8z8lgmz.us-east-1.rds.amazonaws.com",
            user="admin",
            password="AWS$2020",
            port = 3306, #for Mamp users
            database='facebook_ads'
            )

sqlite_select_query = f'''SELECT `COLUMN_NAME` 
FROM `INFORMATION_SCHEMA`.`COLUMNS` 
WHERE `TABLE_SCHEMA`='facebook_ads' 
    AND `TABLE_NAME`='{table_name}';'''

mycursor = mydb.cursor()
mycursor.execute(sqlite_select_query)
order_cols = mycursor.fetchall()
order_cols = [item for sublist in order_cols for item in sublist]
order_cols.remove('state')
mycursor.close()
mydb.close()
mydb = mysql.connector.connect(
            host="database-1.cclpp8z8lgmz.us-east-1.rds.amazonaws.com",
            user="admin",
            password="AWS$2020",
            port = 3306, #for Mamp users
            database='Demo'
            )
mycursor = mydb.cursor()
sqlite_select_query = f"""select {','.join(order_cols)} from {table_name};"""
mycursor.execute(sqlite_select_query)
records = mycursor.fetchall()
mycursor.close()
mydb.close()


mydb = mysql.connector.connect(
            host="database-1.cclpp8z8lgmz.us-east-1.rds.amazonaws.com",
            user="admin",
            password="AWS$2020",
            port = 3306, #for Mamp users
            database='facebook_ads'
            )
mycursor = mydb.cursor()
sql_txt = f"INSERT INTO {table_name}({','.join(order_cols)}) VALUES ({','.join(['%s']*len(records[0]))})"
for record in records:
    try:
        mycursor.execute(sql_txt,record)
    except Exception as e:
        print(e)
    mydb.commit()
mycursor.close()
mydb.close()

InterfaceError: Failed parsing OK packet.

In [ ]:
import sqlite3
import mysql.connector
#get the right order
table_name = 'ads'
mydb = mysql.connector.connect(
            host="database-1.cclpp8z8lgmz.us-east-1.rds.amazonaws.com",
            user="admin",
            password="AWS$2020",
            port = 3306, #for Mamp users
            database='facebook_ads'
            )

sqlite_select_query = f'''SELECT `COLUMN_NAME` 
FROM `INFORMATION_SCHEMA`.`COLUMNS` 
WHERE `TABLE_SCHEMA`='facebook_ads' 
    AND `TABLE_NAME`='{table_name}';'''

mycursor = mydb.cursor()
mycursor.execute(sqlite_select_query)
order_cols = mycursor.fetchall()
order_cols = [item for sublist in order_cols for item in sublist]
order_cols.remove('state')
mycursor.close()
mydb.close()
##
sqliteConnection = sqlite3.connect('FaceBoookADS.db')
cursor = sqliteConnection.cursor()
sqlite_select_query = f"""select {','.join(order_cols)} from {table_name};"""
cursor.execute(sqlite_select_query)
records = cursor.fetchall()
cursor.close()
sqliteConnection.close()

mydb = mysql.connector.connect(
            host="database-1.cclpp8z8lgmz.us-east-1.rds.amazonaws.com",
            user="admin",
            password="AWS$2020",
            port = 3306, #for Mamp users
            database='facebook_ads'
            )
mycursor = mydb.cursor()
sql_txt = f"INSERT INTO {table_name} VALUES ({','.join(['%s']*len(records[0]))})"
for record in records:
    try:
        mycursor.execute(sql_txt,record)
    except Exception as e:
        print(e)
    mydb.commit()
mycursor.close()
mydb.close()



In [ ]:
import sqlite3
import mysql.connector
#get the right order
table_name = 'search_term'
mydb = mysql.connector.connect(
            host="database-1.cclpp8z8lgmz.us-east-1.rds.amazonaws.com",
            user="admin",
            password="AWS$2020",
            port = 3306, #for Mamp users
            database='facebook_ads'
            )

sqlite_select_query = f'''SELECT `COLUMN_NAME` 
FROM `INFORMATION_SCHEMA`.`COLUMNS` 
WHERE `TABLE_SCHEMA`='facebook_ads' 
    AND `TABLE_NAME`='{table_name}';'''

mycursor = mydb.cursor()
mycursor.execute(sqlite_select_query)
order_cols = mycursor.fetchall()
order_cols = [item for sublist in order_cols for item in sublist]
order_cols.remove('id')
mycursor.close()
mydb.close()

mydb = mysql.connector.connect(
            host="database-1.cclpp8z8lgmz.us-east-1.rds.amazonaws.com",
            user="admin",
            password="AWS$2020",
            port = 3306, #for Mamp users
            database='Demo'
            )
mycursor = mydb.cursor()
sqlite_select_query = f"""select {','.join(order_cols)} from {table_name};"""
mycursor.execute(sqlite_select_query)
records1 = mycursor.fetchall()
mycursor.close()
mydb.close()
sqliteConnection = sqlite3.connect('FaceBoookADS.db')
cursor = sqliteConnection.cursor()
sqlite_select_query = f"""select {','.join(order_cols)} from {table_name};"""
cursor.execute(sqlite_select_query)
records2 = cursor.fetchall()
cursor.close()
sqliteConnection.close()

records = list(set(records1+records2))

mydb = mysql.connector.connect(
            host="database-1.cclpp8z8lgmz.us-east-1.rds.amazonaws.com",
            user="admin",
            password="AWS$2020",
            port = 3306, #for Mamp users
            database='facebook_ads'
            )
mycursor = mydb.cursor()
sql_txt = f"INSERT INTO {table_name} VALUES ({','.join(['%s']*len(records[0]))})"
for record in records:
    try:
        mycursor.execute(sql_txt,record)
    except Exception as e:
        print(e)
    mydb.commit()
mycursor.close()
mydb.close()


In [17]:

sqliteConnection = sqlite3.connect('FaceBoookADS.db')
cursor = sqliteConnection.cursor()
sqlite_select_query = f"""select * from ads;"""
cursor.execute(sqlite_select_query)
records2 = cursor.fetchall()
cursor.close()
sqliteConnection.close()

In [ ]:
mydb = mysql.connector.connect(
            host="database-1.cclpp8z8lgmz.us-east-1.rds.amazonaws.com",
            user="admin",
            password="AWS$2020",
            port = 3306, #for Mamp users
            database='demo'
            )
mycursor = mydb.cursor()
sqlite_select_query = f"""select * from ads;"""
mycursor.execute(sqlite_select_query)
records1 = mycursor.fetchall()
mycursor.close()
mydb.close()

sqliteConnection = sqlite3.connect('FaceBoookADS.db')
cursor = sqliteConnection.cursor()
sqlite_select_query = f"""select * from ads;"""
cursor.execute(sqlite_select_query)
records2 = cursor.fetchall()
cursor.close()
sqliteConnection.close()

records = list(set(records1+records2))

mydb = mysql.connector.connect(
            host="database-1.cclpp8z8lgmz.us-east-1.rds.amazonaws.com",
            user="admin",
            password="AWS$2020",
            port = 3306, #for Mamp users
            database='facebook_ads'
            )
mycursor = mydb.cursor()
sql_txt = f"INSERT INTO ads VALUES ({','.join(['%s']*len(records[0]))})"
for record in records:
    try:
        mycursor.execute(sql_txt,record)
    except Exception as e:
        print(e)
    mydb.commit()
mycursor.close()
mydb.close()

In [45]:
sqlite_select_query == f"""select static_ID,page_name,favorites,country,block,profile_pic,Facebook_ID,Page_likes,instgram_ID,insta_followers,Ads_count,block_date from facebook_pages;"""

True

In [ ]:
mydb = mysql.connector.connect(
            host="database-1.cclpp8z8lgmz.us-east-1.rds.amazonaws.com",
            user="admin",
            password="AWS$2020",
            port = 3306, #for Mamp users
            database='demo'
            )
mycursor = mydb.cursor()
sqlite_select_query = f"""select * from search_terms;"""
mycursor.execute(sqlite_select_query)
records1 = mycursor.fetchall()
mycursor.close()
mydb.close()

sqliteConnection = sqlite3.connect('FaceBoookADS.db')
cursor = sqliteConnection.cursor()
sqlite_select_query = f"""select * from search_terms;"""
cursor.execute(sqlite_select_query)
records2 = cursor.fetchall()
cursor.close()
sqliteConnection.close()

records = list(set(records1+records2))

mydb = mysql.connector.connect(
            host="database-1.cclpp8z8lgmz.us-east-1.rds.amazonaws.com",
            user="admin",
            password="AWS$2020",
            port = 3306, #for Mamp users
            database='facebook_ads'
            )
mycursor = mydb.cursor()
sql_txt = f"INSERT INTO search_terms VALUES ({','.join(['%s']*len(records[0]))})"
for record in records:
    try:
        mycursor.execute(sql_txt,record)
    except Exception as e:
        print(e)
    mydb.commit()
mycursor.close()
mydb.close()

## fix one time broken ads 

In [75]:
import sqlite3
sqliteConnection = sqlite3.connect('FaceBoookADS.db')
cursor = sqliteConnection.cursor()
sqlite_select_query = f"""select * from "ads" WHERE "cumulative_ads_count" LIKE '%2022%';"""
cursor.execute(sqlite_select_query)
records = cursor.fetchall()

# records = [i[0] for i in records]
print(len(records))
for record in records:
    data = []
    data.append(record[-4])
    data += (list(record[5:-4]))
    data.append(record[0])
    cursor.execute('''UPDATE ads SET
videos_length = ?,content = ? ,Footer_text = ? ,Footer_action = ? ,Page_name = ? ,AD_occurance = ? ,Facebook_ID = ? ,Page_likes = ? ,instgram_ID = ? ,insta_followers = ? ,static_ID = ? ,Ads_count = ? ,cumulative_ads_count = ? ,days = ? ,date = ? ,hits = ? ,search_term = ?
where AD_ID = ?'''
  ,data)
sqliteConnection.commit()
sqliteConnection.close()

10


In [ ]:
cursor.execute('''UPDATE ads SET
video_length = ?,content = ? ,Footer_text = ? ,Footer_action = ? ,Page_name = ? ,AD_occurance = ? ,Facebook_ID = ? ,Page_likes = ? ,instgram_ID = ? ,insta_followers = ? ,static_ID = ? ,Ads_count = ? ,cumulative_ads_count = ? ,days = ? ,date = ? ,hits = ? ,search_term = ?
where AD_ID = ?'''
  ,data)
conn.commit()
conn.close()

('108298711454326',
 'alaa_.888',
 1,
 0,
 1,
 '2022-10-17 02:35:22.770894+00:00',
 'NO Facebook ID found',
 2000,
 '@alaa_.888',
 0,
 'https://scontent.cdninstagram.com/v/t51.2885-15/180843381_1932820030189617_3599037007407971571_n.jpg?stp=dst-jpg_s200x200&_nc_cat=104&ccb=1-7&_nc_sid=8ae9d6&_nc_ohc=Oi93nOfTAIMAX-FwPEz&_nc_ht=scontent.cdninstagram.com&oh=00_AT9IdPEYUXBOJ85GWt0uNyNHjFG_QNy2xu6C0KbV8syvOQ&oe=634CB406',
 'ALL')

### databasse migration

In [22]:
records1[0]

('150398005058111',
 'ASD Account Financial Advisor SA',
 '0',
 '0',
 '0',
 None,
 'NO Facebook ID found',
 '0',
 'NO insta ID found',
 '0',
 'https://scontent-hbe1-1.xx.fbcdn.net/v/t39.35426-6/298281900_5852012538182642_4454294096832948711_n.jpg?stp=dst-jpg_s60x60&_nc_cat=111&ccb=1-7&_nc_sid=cf96c8&_nc_ohc=Iqz8uiZhDGIAX8ZgzGN&_nc_ht=scontent-hbe1-1.xx&oh=00_AT_H7coqy-qPa_Hyba1Xyhgs3De9Cwn0waaBHyNoKXYMsw&oe=635EAD44',
 'ALL')

In [4]:
import sqlite3
sqliteConnection = sqlite3.connect('FaceBoookADS.db')
cursor = sqliteConnection.cursor()
sqlite_select_query = f"""select * from facebook_pages;"""
cursor.execute(sqlite_select_query)
records = cursor.fetchall()

# records = [i[0] for i in records]
print(len(records))
cursor.close()
sqliteConnection.close()

4440


In [5]:
mydb = mysql.connector.connect(
            host="database-1.cclpp8z8lgmz.us-east-1.rds.amazonaws.com",
            user="admin",
            password="AWS$2020",
            port = 3306, #for Mamp users
            database='Demo'
            )
mycursor = mydb.cursor()
for record in records:
    mycursor.execute("INSERT INTO facebook_pages_clone VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)",record)
    mydb.commit()
mydb.close()
mycursor.close()
    # try:
# mycursor.executemany("""INSERT INTO facebook_pages_clone VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)""", records);
# mydb.commit()

KeyboardInterrupt: 

In [88]:
import mysql.connector
mydb = mysql.connector.connect(
host="database-1.cclpp8z8lgmz.us-east-1.rds.amazonaws.com",
user="admin",
password="AWS$2020",
port = 3306, #for Mamp users
database='Demo'
)

In [81]:
sqliteConnection = sqlite3.connect('FaceBoookADS.db')
cursor = sqliteConnection.cursor()
sqlite_select_query = f"""select * from facebook_pages limit 20;"""
cursor.execute(sqlite_select_query)
records = cursor.fetchall()

# records = [i[0] for i in records]
print(len(records))

sqliteConnection.close()

20


In [18]:
records2

[('1218107492378490',
  '2022-10-19',
  'https://scontent-hbe1-1.xx.fbcdn.net/v/t39.35426-6/255415194_321543819365310_8369304891521995374_n.jpg?stp=dst-jpg_s60x60&_nc_cat=100&ccb=1-7&_nc_sid=cf96c8&_nc_ohc=3og-PlurviwAX-ufe1P&_nc_ht=scontent-hbe1-1.xx&oh=00_AT-FbEodsDL5V8jZVvTZdWrlaUDnh7jAeuJI7ZT-DNvTqA&oe=635D5E11',
  'https://www.instagram.com/hmns.id/?fbclid=IwAR3zrsAr5CumbSwIa_8aF_pYhiae0WB5R6q3zLZT--9MUUtkG226vG3tecE',
  'https://video-hbe1-1.xx.fbcdn.net/v/t42.1790-2/254846199_469273874510655_834289410278840730_n.?_nc_cat=108&ccb=1-7&_nc_sid=cf96c8&_nc_ohc=GsLAl-fhCEoAX-4wti2&_nc_ht=video-hbe1-1.xx&oh=00_AT_3r5ezIUdLU5FnpbbVGd_irlJqtwZmpqoyvtlMEWgaIQ&oe=6357E2F1',
  '16',
  'EOS membawa solar accord yaitu "the smell of the sun" yang memberikan kesan radiant & hangat seperti liburan di musim panas di Bali. Click "LEARN MORE" now.',
  'INSTAGRAM.COM\nMeet Essence of the Sun!\nLearn More',
  'Learn More',
  'HMNS Perfume',
  1,
  'ID: 2684857714919929',
  6000,
  '@hmns.id',
  2000,

In [82]:
sqliteConnection = sqlite3.connect('FaceBoookADS.db')
cursor = sqliteConnection.cursor()
sqlite_select_query = f"""select * from search_terms limit 20;"""
cursor.execute(sqlite_select_query)
records = cursor.fetchall()

# records = [i[0] for i in records]
print(len(records))

sqliteConnection.close()

1


In [48]:
2,[1,5,6]

(2, [1, 5, 6])

In [6]:
records[0]

('775334143532273',
 '2022-10-24',
 'https://scontent-hbe1-1.xx.fbcdn.net/v/t39.35426-6/310693240_800679937829605_2452436212821073899_n.jpg?stp=dst-jpg_s60x60&_nc_cat=105&ccb=1-7&_nc_sid=cf96c8&_nc_ohc=sFpHLosVFTsAX99YEZN&_nc_ht=scontent-hbe1-1.xx&oh=00_AT_vEqMJQN5Rs9k3BVenSTj0TbllMOJi5tfbIcWpZtz5-A&oe=635A72CF',
 '',
 'https://video-hbe1-1.xx.fbcdn.net/v/t42.1790-2/309370745_503324988100209_3490100667989017185_n.?_nc_cat=109&ccb=1-7&_nc_sid=cf96c8&_nc_ohc=Q_tejMME6tsAX8Fn5aX&_nc_ht=video-hbe1-1.xx&oh=00_AT-c_BhbardfSjlmpDFmkjT6C99aUuv7_AFlvvrMIdL2aQ&oe=63569E83',
 'Original Testers 💯\n👉 DM for orders 👈\n\n100ml\n50% Discount 🔥\nBatch Code + Serial Number 👌\n.\n.\n.\n#original #testers #testerparfum\n#strongerwithyou #versaceeros #mancera\n#sauvage #اوتليت #sauvagedior #تومفورد #تومفوردبلاكاوركيد #tomfordblackorchid #tomfordombreleather #tomfordmen\n#1million #perfume #perfumes #fragrance #fragrances #bleuchanel #bleuchanelparfum #invictus #boss #hugoboss #hugo #sìpassione #sipassione 

In [56]:
list1 = (a,[b,c,d]) = 2,[1,5,6]

In [57]:
list1

(2, [1, 5, 6])

## remove not downloaded ads 

In [97]:
# def get_records(SLQQuery):
    sqliteConnection = sqlite3.connect('FaceBoookADS.db')
    cursor = sqliteConnection.cursor()
    cursor.execute(SLQQuery)
    records = cursor.fetchall()
#     print(len(records))
    sqliteConnection.close()
    return records

In [ ]:
SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE='BASE TABLE'


In [52]:
b

1

In [69]:
# sqlite_select_query = f"""SELECT DISTINCT static_id,page_name, FROM ads where page_name is not '' """
# static_ids = [i[0] for i in get_records(sqlite_select_query)] 
# static_ids.remove('NO static ID found')
# # records = [i[0] for i in records]
# # print(len(records))
# # for record in records:
# #     try:
# #         cursor.execute('''INSERT INTO facebook_pages VALUES 
# #         (?, ?,?)'''
# #                   , (record[0],record[1],False))
# #         sqliteConnection.commit()
# #     except:
# #         pass

4699


### create pages_datebase



In [70]:
sqlite_select_query = f"""SELECT DISTINCT static_id FROM ads where page_name is not '' """
sqliteConnection = sqlite3.connect('FaceBoookADS.db')
cursor = sqliteConnection.cursor()
cursor.execute(sqlite_select_query)
records = cursor.fetchall()
sqliteConnection.close()
static_ids = [i[0] for i in records] 
static_ids.remove('NO static ID found')
for static_id in static_ids:
    sqliteConnection = sqlite3.connect('FaceBoookADS.db')
    cursor = sqliteConnection.cursor()
    sqlite_select_query = f"""SELECT ads_count,Facebook_ID,insta_followers,instgram_ID,Page_likes,profile_pic,page_name FROM ads where static_id = {static_id} order by date desc limit 1"""
    cursor.execute(sqlite_select_query)
    records = cursor.fetchall()
    ads_count,Facebook_ID,insta_followers,instgram_ID,Page_likes,profile_pic,page_name  = records[0]
    cursor.close()
    sqliteConnection.close()
    try:
        sqliteConnection = sqlite3.connect('FaceBoookADS.db')
        cursor = sqliteConnection.cursor()
        cursor.execute(f"""insert into facebook_pages(static_ID,ads_count,Facebook_ID,insta_followers,instgram_ID,Page_likes,profile_pic,page_name,favorites,block) VALUES (?,?,?,?,?,?,?,?,0,0)""",(static_id,ads_count,Facebook_ID,insta_followers,instgram_ID,Page_likes,profile_pic,page_name))
        sqliteConnection.commit()
        sqliteConnection.close()
    except:
        sqliteConnection = sqlite3.connect('FaceBoookADS.db')
        cursor = sqliteConnection.cursor()
        cursor.execute(f"""
        update facebook_pages set 
        Ads_count = ?,
        Facebook_ID = ?,
        insta_followers = ?,
        instgram_ID = ?,
        Page_likes = ?,
        profile_pic = ?,
        page_name = ?
        where static_id = ?
        """,(ads_count,Facebook_ID,insta_followers,instgram_ID,Page_likes,profile_pic,page_name,static_id))
        sqliteConnection.commit()
        sqliteConnection.close()
#clean_old_pages 
sqlite_select_query = f"""DELETE FROM facebook_pages where Ads_count is NULL """
#DELETE FROM table
# WHERE search_condition;
sqliteConnection = sqlite3.connect('FaceBoookADS.db')
cursor = sqliteConnection.cursor()
cursor.execute(sqlite_select_query)
sqliteConnection.commit()
# records = cursor.fetchall()
sqliteConnection.close()

4692


In [8]:
'%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s' == ','.join(['%s']*12)

True

## solve no data problem 

In [10]:
'INSERT INTO facebook_pages VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)' =="INSERT INTO facebook_pages VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"

True

## count

In [162]:
sqliteConnection = sqlite3.connect('FaceBoookADS.db')
cursor = sqliteConnection.cursor()
cursor.execute("""select static_ID, count(*) as c FROM Ads GROUP BY static_ID""")
records = cursor.fetchall()
#     print(len(records))
sqliteConnection.close()

In [163]:
records

[('0', 1),
 ('100100119547239', 1),
 ('100101471794502', 2),
 ('100106772530451', 1),
 ('100109449544319', 1),
 ('100109651405241', 1),
 ('100110425921069', 13),
 ('100111301813165', 2),
 ('100112392883547', 3),
 ('100113002863377', 2),
 ('100116466071014', 5),
 ('100118665230213', 1),
 ('100119601914982', 2),
 ('100119731969906', 1),
 ('100121389440517', 1),
 ('100123525776787', 1),
 ('100124119232558', 1),
 ('100124741443742', 5),
 ('100125292366804', 1),
 ('100127202656038', 2),
 ('100128456218462', 2),
 ('100129335253816', 1),
 ('100132049394692', 1),
 ('100135505645208', 3),
 ('100136379150137', 1),
 ('100136521449570', 1),
 ('100136819540297', 1),
 ('100138959552683', 1),
 ('100147186159541', 2),
 ('100150285649670', 2),
 ('100151212798459', 1),
 ('100154388439386', 1),
 ('100154432044067', 1),
 ('100155896134144', 1),
 ('100159685363036', 2),
 ('100160619446323', 1),
 ('1001619056584927', 1),
 ('100164375526308', 2),
 ('100166622763048', 1),
 ('100170209367620', 1),
 ('100171215

In [87]:
!pip install mysql.connector

     ---------------------------------------- 11.9/11.9 MB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for mysql.connector: filename=mysql_connector-2.2.9-cp39-cp39-win_amd64.whl size=247946 sha256=c55ac5566bcf3136d7d3a7d18cce1caeb8f0de1a3af0b40a56f42cbed0bc774e
  Stored in directory: c:\users\asd_v\appdata\local\pip\cache\wheels\7b\14\39\5aad423666e827dfe9a1fbcd111ac17171e7c9865d570780ce
Successfully built mysql.connector
